In [65]:
import traci
import time
import random
import numpy as np
import csv
import os
import random, numpy as np
import pandas as pd, matplotlib.pyplot as plt

SEED = 8
random.seed(SEED)
np.random.seed(SEED)
# Imports and configuration
# Use sumo or sumo-gui depending on your preference
SUMO_BINARY  = "sumo-gui"  # or "sumo-gui"
SUMO_CFG  = "./sumo_cfg/simulation.sumocfg"  # your .sumocfg file
SUMO_STATE = "./sumo_cfg/simulation_state.xml"  # your initial state file
DE_RESULTS_CSV = "./results/de_results.csv"
DEFAULT_RESULTS_CSV = "./results/default_results.csv"
SEED_DE_RESULTS_CSV = "./results/seed_de_results.csv"

# Global timing constants
CLEAR_MAXIMUM_QUEUE_W, FAIRNESS_W = 0.7, 0.3  # prioritize queue clearance
CYCLE_LENGTH_DEFAULT = 90   # s
MAX_CYCLE_LENGTH = 150      # s
MIN_CYCLE_LENGTH = 70      # s
LOST_TIME = 12              # s (amber + all-red total)
GREEN_MIN = 15              # s per direction

In [77]:
traci.close()

In [78]:
traci.start([SUMO_BINARY, "-c", SUMO_CFG])

# Run simulation for 100 steps
for step in range(100):
    traci.simulationStep()

    # Get traffic light IDs
    tls_ids = traci.trafficlight.getIDList()
    for tls_id in tls_ids:
        phase = traci.trafficlight.getPhase(tls_id)

traci.close()

 Retrying in 1 seconds


In [66]:
#Intersection metadata
# Define lane groups for metric aggregation
NS_LANES = ["1200728225#1_0", "1200728225#1_1", "1221994726#0_0", "1221994726#0_1"]  # replace with your real lane IDs
EW_LANES = ["1265822568#3_0", "1265822568#3_1", "1265822568#3_2"]

# Your target traffic light ID
TL_ID = "cluster_13075564400_13075589603_411926344"  # replace with your actual traffic light id in SUMO net

In [67]:
def apply_plan(tl_id, g_main, g_cross, amber=3, all_red=3):
    """
    Define a 2-phase signal plan (NS and EW) with amber and all-red times.
    """
    amrber_red_phase_duration = amber + all_red
    phases = [
        traci.trafficlight.Phase(g_main, "rrrrrGGggGGG"),   # NS green
        traci.trafficlight.Phase(amrber_red_phase_duration, "rrrrryyyyyyy"),    # NS amber
        traci.trafficlight.Phase(g_cross, "GGGGGrrrrrrr"),  # EW green
        traci.trafficlight.Phase(amrber_red_phase_duration, "yyyyyrrrrrrr"),    # EW amber
    ]

    logic = traci.trafficlight.Logic("custom_logic", 0, 0, phases)
    traci.trafficlight.setCompleteRedYellowGreenDefinition(tl_id, logic)


In [69]:
# Metrics calculation
def cycle_metrics(ns_lanes, ew_lanes, steps):
    q_ns_max = q_ew_max = 0
    ns_wait_total = ew_wait_total = ns_veh = ew_veh = 0

    for _ in range(steps):
        traci.simulationStep()
        q_ns = sum(traci.lane.getLastStepHaltingNumber(l) for l in ns_lanes)
        q_ew = sum(traci.lane.getLastStepHaltingNumber(l) for l in ew_lanes)
        q_ns_max = max(q_ns_max, q_ns)
        q_ew_max = max(q_ew_max, q_ew)

        for v in traci.vehicle.getIDList():
            wt = traci.vehicle.getWaitingTime(v)
            lane = traci.vehicle.getLaneID(v)
            if lane in ns_lanes:
                ns_wait_total += wt
                ns_veh += 1
            elif lane in ew_lanes:
                ew_wait_total += wt
                ew_veh += 1

    avg_delay_ns = ns_wait_total / max(ns_veh, 1)
    avg_delay_ew = ew_wait_total / max(ew_veh, 1)
    fairness = (avg_delay_ns - avg_delay_ew) ** 2  # variance proxy

    O1 = max(q_ns_max, q_ew_max)
    O2 = fairness
    return O1, O2


In [70]:
def get_green_split(s, C):
    g_main = max(GREEN_MIN, round((C - LOST_TIME) * s))
    g_cross = max(GREEN_MIN, (C - LOST_TIME) - g_main)
    return g_main, g_cross

In [71]:
# Evaluation function
def evaluate(s, C):
    # Restore identical starting conditions before evaluating this candidate
    traci.simulation.loadState(SUMO_STATE)
    g_main, g_cross = get_green_split(s, C)
    apply_plan(TL_ID, g_main, g_cross)
    steps = int(C)
    O1, O2 = cycle_metrics(NS_LANES, EW_LANES, steps)

    return O1, O2


In [72]:
def differential_evolution(elite_last=None, time_budget_s=15,
                           pop_size=12, F=0.6, CR=0.9):
    """
    Differential Evolution (DE/rand/1/bin) for 2 parameters: (s, C)
    Optimizes two objectives (O1, O2) within a strict time budget.
    """

    start = time.perf_counter()
    bounds = [(0.2, 0.8), (MIN_CYCLE_LENGTH, MAX_CYCLE_LENGTH)]  # (s_min, s_max), (C_min, C_max)

    # --- Initialize population
    pop = [np.array([random.uniform(*b) for b in bounds]) for _ in range(pop_size)]
    scores = [None] * pop_size

    # Include previous elite as first individual (temporal continuity)
    if elite_last is not None:
        pop[0] = np.array([elite_last[0], elite_last[1]])

    def evaluate_candidate(x):
        s, C = float(x[0]), float(x[1])
        return evaluate(s, C)

    # --- Evaluate initial population
    for i in range(pop_size):
        O1, O2 = evaluate_candidate(pop[i])
        scores[i] = (O1, O2)

    # --- Evolution loop (bounded by wall-clock)
    while time.perf_counter() - start < time_budget_s:
        for i in range(pop_size):
            # Mutation: pick three distinct indices ≠ i
            idxs = list(range(pop_size))
            idxs.remove(i)
            r1, r2, r3 = random.sample(idxs, 3)

            # DE/rand/1
            mutant = pop[r1] + F * (pop[r2] - pop[r3])

            # Ensure bounds
            for j in range(len(bounds)):
                mutant[j] = np.clip(mutant[j], bounds[j][0], bounds[j][1])

            # Crossover
            trial = np.copy(pop[i])
            for j in range(len(bounds)):
                if random.random() < CR:
                    trial[j] = mutant[j]

            # Evaluate trial
            O1_t, O2_t = evaluate_candidate(trial)
            O1_i, O2_i = scores[i]

            # --- Selection
            # Combine both objectives using scalarized score (can adjust weight)
            score_t = CLEAR_MAXIMUM_QUEUE_W * O1_t + FAIRNESS_W * O2_t
            score_i = CLEAR_MAXIMUM_QUEUE_W * O1_i + FAIRNESS_W * O2_i

            if score_t <= score_i:
                pop[i], scores[i] = trial, (O1_t, O2_t)

        # Break early if time budget exceeded
        if time.perf_counter() - start > time_budget_s:
            break

    # --- Return the best individual
    best_idx = min(range(pop_size), key=lambda i: scores[i][0] + scores[i][1])
    s_best, C_best = pop[best_idx]
    O1_best, O2_best = scores[best_idx]

    return (s_best, C_best, O1_best, O2_best)


In [73]:
import os
def seed_de_simulation():
    if  os.path.exists(SEED_DE_RESULTS_CSV):
        os.remove(SEED_DE_RESULTS_CSV)
        print(f"Old results file {SEED_DE_RESULTS_CSV} deleted.")
    traci.start([
        SUMO_BINARY,
        "-c", SUMO_CFG,
        "--seed", str(SEED),
        "--random", "false"   # ensure it uses the given seed deterministically
    ])
    elite = None

    with open(SEED_DE_RESULTS_CSV, "w", newline="") as f:
        writer = csv.writer(f)
        writer
        writer.writerow(["cycle", "s", "C", "O1", "O2"])

        for cycle in range(80):  # run 20 cycles for demo     
            print(f"\n=== Cycle {cycle+1} ===")
             # 1. Save the current live SUMO state once
            traci.simulation.saveState(SUMO_STATE)

            # 2 Run DE using that snapshot as baseline
            best = differential_evolution(elite_last=elite, time_budget_s=15)
            s, C, O1, O2 = best
            elite = (s, C)

            # 3️⃣ Apply only the *best plan* to the live SUMO world
            g_main, g_cross = get_green_split(s, C)
            apply_plan(TL_ID, g_main, g_cross)

            for _ in range(int(C)):
                traci.simulationStep()

            print(f"Chosen split={s:.2f}, C={C:.1f}, O1={O1:.2f}, O2={O2:.2f}")
            writer.writerow([cycle+1, s, C, O1, O2])
            f.flush()
    traci.close()   

seed_de_simulation()

 Retrying in 1 seconds

=== Cycle 1 ===


/tmp/ipykernel_2266/713683444.py:14: UserWarning: Call to deprecated function setProgramLogic, use setCompleteRedYellowGreenDefinition instead.
  traci.trafficlight.setCompleteRedYellowGreenDefinition(tl_id, logic)


Chosen split=0.47, C=109.6, O1=18.00, O2=4.03

=== Cycle 2 ===
Chosen split=0.47, C=109.6, O1=30.00, O2=3.85

=== Cycle 3 ===
Chosen split=0.36, C=103.9, O1=31.00, O2=0.12

=== Cycle 4 ===
Chosen split=0.34, C=102.8, O1=36.00, O2=0.10

=== Cycle 5 ===
Chosen split=0.31, C=147.6, O1=36.00, O2=0.07

=== Cycle 6 ===
Chosen split=0.64, C=126.7, O1=28.00, O2=6.66

=== Cycle 7 ===
Chosen split=0.42, C=145.3, O1=35.00, O2=0.72

=== Cycle 8 ===
Chosen split=0.42, C=146.7, O1=36.00, O2=7.75

=== Cycle 9 ===
Chosen split=0.27, C=136.3, O1=34.00, O2=20.51

=== Cycle 10 ===
Chosen split=0.67, C=149.4, O1=32.00, O2=0.04

=== Cycle 11 ===
Chosen split=0.33, C=93.5, O1=26.00, O2=0.59

=== Cycle 12 ===
Chosen split=0.44, C=83.5, O1=36.00, O2=13.53

=== Cycle 13 ===
Chosen split=0.62, C=148.7, O1=35.00, O2=0.13

=== Cycle 14 ===
Chosen split=0.42, C=145.6, O1=35.00, O2=10.59

=== Cycle 15 ===
Chosen split=0.22, C=88.6, O1=36.00, O2=0.29

=== Cycle 16 ===
Chosen split=0.42, C=118.0, O1=33.00, O2=3.47

=

FatalTraCIError: Connection closed by SUMO.

In [ ]:
def visualize_results():
    df = pd.read_csv("results.csv")

    plt.plot(df["cycle"], df["O1"], label="Max Queue")
    plt.plot(df["cycle"], df["O2"], label="Fairness")
    plt.legend(); plt.xlabel("Cycle"); plt.ylabel("Objective"); plt.show()